In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append(os.path.join(os.getcwd(), "src"))

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import importlib
import time

from util import utils
from util import plotting
from stats import hbStats

### Load and prepare data

In [ ]:
dataFolder = "../../scripts_data/fitbit_analyzer/dataDumpSubset"

In [ ]:
start = time.time()
hbData = utils.loadHBData(dataFolder)
end = time.time()
print("Data loaded in {:.2f}s".format(end - start))
data = pd.concat(hbData, ignore_index=True)
print("Sample")
print(data.head())

### Basic Overall Stats

In [ ]:
hbStats.basicStats(data)

### Basic Aggregated Stats

In [ ]:
hourStats = hbStats.groupByBasicStats(data['datetime'].dt.hour, data, 'hour')
plotting.plot(hourStats, ['count', 'max', 'min', 'std'], 'hour', 2, 2)
#dayStats = hbStats.groupByBasicStats(data['datetime'].dt.day, data, 'day')
#plotting.plot(dayStats, ['count', 'max', 'min', 'std'], 'day', 2, 2)
monthStats = hbStats.groupByBasicStats(data['datetime'].dt.month, data, 'month')
plotting.plotMonthlyStatsHb(monthStats)
dayOfWeekStats = hbStats.groupByBasicStats(data['datetime'].dt.dayofweek, data, 'weekday')
plotting.plotWeekdayStatsHb(dayOfWeekStats)

### Daily Plot

In [ ]:
nData = data.set_index(["datetime"])
print(nData.head())
#print(data[data['datetime'].dt.date==data['datetime'][0].date()].mean())
#nData.resample('1D').mean().plot()
resampledData = nData.resample('1D')['value'].agg({'max' : np.max, 'min' : np.min,
                                           'mean' : np.mean}).reset_index()
meltedData = pd.melt(resampledData, id_vars=['datetime'], value_vars=['max', 'mean', 'min'],
                    var_name='var', value_name='value')
print(meltedData.head())
#resampledData.plot()
fig, ax = plt.subplots()
sns.pointplot(data=meltedData, x='datetime', y='value', hue='var', ax=ax)
from matplotlib.ticker import LinearLocator
from matplotlib.ticker import FuncFormatter
import matplotlib.dates as dates

def myFormatter(x, pos):
    return pd.to_datetime(x)

#ax.get_yaxis().set_major_locator(LinearLocator(numticks=5))
#ax.xaxis.set_major_locator(dates.AutoDateLocator())
#ax.xaxis.set_major_formatter(dates.DateFormatter('%Y.%m.%d'))
#sns.plt.gca().xaxis.set_major_locator(FuncFormatter(myFormatter))
#sns.plt.gca().xaxis.set_major_formatter(dates.DateFormatter('%Y.%m.%d'))
sns.plt.gca().xaxis.set_major_formatter(FuncFormatter(myFormatter))
# put the labels at 45deg since they tend to be too long
#fig.autofmt_xdate()
#ax.set_xticklabels(xticks, rotation=45)
sns.plt.gcf().autofmt_xdate()
sns.plt.show()

In [ ]:
plotting.plotDailyStatsHb(data)

### Min/Max Values

In [ ]:
hbStats.getMaxValues(data, 5)

In [ ]:
hbStats.getMinValues(data, 5)